In [233]:
# imports
import numpy as np
import pandas as pd
import os
import requests
import foursquare
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# import langitude and latitude coordinates from Part 1 City bike acitivity and store in a Dataframe
city_bike_coord = pd.read_csv('../data/CityBike_longlat.csv')
city_bike_coord = city_bike_coord.round(2)
city_bike_coord

In [ ]:
# get each pair of long lat coordinates
long_lat_strings = []

for index, row in city_bike_coord.iterrows():
  # Extract the longitude and latitude values
  longitude = row['longitude']
  latitude = row['latitude']

  # Combine them into a tuple
  long_lat_pair = f"{longitude},{latitude}"

  long_lat_strings.append(long_lat_pair)

long_lat_strings

In [ ]:
# API URL and API token stored as an environment variable
client_access_token = os.environ['FOURSQUARE_AUTH_TOKEN']

# get response in json while passing API token in the header
headers = {
    "Accept": "application/json",
    "Authorization": client_access_token
}

responses = []

for long_lat_string in long_lat_strings:
    #URL
    
    longitude, latitude = long_lat_string.split(",")
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius=1000&query=restaurant"
    response = requests.get(url, headers=headers)
    data = response.json()
    responses.append(data)

print(responses)


In [ ]:
# Flatten responses by two levels to get results
flat_responses=[]
for level_one in responses:
    for level_two in level_one['results']:
        flat_responses.append(level_two)
flat_responses



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [311]:
# create a new dataframe we will append to
results =[]


#iterate through results "data['results]" will give iterate through json data from one level down
for result in flat_responses:
    name = result['name']
    location = result['location']

    categories = []
    for category in result['categories']:
        categories.append(category['name'])

    result_dict = {
        'name': name,
        'location': location,
        'categories': categories
    }
    # add respective POI in list object
    results.append(result_dict)

In [312]:
len(results)

1334

Put your parsed results into a DataFrame

In [327]:
# store results in dataframe
df = pd.DataFrame(results)
df_name = df['name']
df_location = pd.json_normalize(df['location'])
df_categories = df['categories']

merged_df = pd.merge(df_name, df_location,left_index=True, right_index=True,how='inner')
final_merge_df = pd.merge(merged_df, df_categories,left_index=True, right_index=True, how='inner')
final_merge_df

,name,address,country,cross_street,formatted_address,locality,postcode,region,address_extended,categories
0,La Luna Restaurant Downtown,306 King St W,CA,at Queen St,"306 King St W (at Queen St), Hamilton ON L8P 1B1",Hamilton,L8P 1B1,ON,NaN,"[Lebanese Restaurant, Mediterranean Restaurant..."
1,Coop Hamilton,274 King St W,CA,,"274 King St W, Hamilton ON L8P 1J6",Hamilton,L8P 1J6,ON,# 272,[Fried Chicken Joint]
2,Smoke's Poutinerie,112 George St,CA,at Queen St. S.,"112 George St (at Queen St. S.), Hamilton ON L...",Hamilton,L8P 1E2,ON,NaN,"[Fast Food Restaurant, Poutine Restaurant, Ste..."
3,Coop Wicked Chicken Hamilton,274 King St W,CA,Hess St,"274 King St W (Hess St), Hamilton ON L8P 1J6",Hamilton,L8P 1J6,ON,# 272,[Fried Chicken Joint]
4,Vida la Pita,217 King St W,CA,Caroline,"217 King St W (Caroline), Hamilton ON L8P 1A7",Hamilton,L8P 1A7,ON,NaN,"[Fast Food Restaurant, Middle Eastern Restaurant]"
...,...,...,...,...,...,...,...,...,...,...
1329,Hambrgr,207 Ottawa St N,CA,,"207 Ottawa St N, Hamilton ON L8H 3Z4",Hamilton,L8H 3Z4,ON,NaN,"[Bar, Burger Joint]"
1330,Cannon Coffee Co,179 Ottawa St N,CA,Cannon Street,"179 Ottawa St N (Cannon Street), Hamilton ON L...",Hamilton,L8H 3Z4,ON,NaN,"[Café, Coffee Shop, Restaurant]"
1331,Boston Pizza,1219 Barton St E,CA,Kenilworth Ave. N.,"1219 Barton St E (Kenilworth Ave. N.), Hamilto...",Hamilton,L8H 2V4,ON,NaN,[Pizzeria]
1332,Hi-Line Centre Pizza & Wings,1150 King St E,CA,Gage,"1150 King St E (Gage), Hamilton ON L8M 1E8",Hamilton,L8M 1E8,ON,# 2,[Pizzeria]


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating